# 패키지 불러오기

In [1]:
!openai --version

openai 1.35.7


In [2]:
import os
import time
import yaml
import re
import json

from openai import OpenAI
import pandas as pd
import ipywidgets as widgets # interactive display
from tqdm import tqdm # progress bar
from dotenv import load_dotenv

# ENV, Config 파일 읽기

In [3]:
# .env 파일에서 환경 변수 로드(API 키)
load_dotenv()

True

In [4]:
# YAML 파일 열기
yaml_path = 'config.yaml' # todo: config 파일과 합치기

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# 인스턴스 생성

In [5]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 함수 정의

In [6]:
def get_response(messages):
    """
    model: 모델 종류
    messages: 사용자의 입력과 모델의 출력을 교환하는 메시지 목록
    max_tokens: 생성될 응답의 최대 길이
    temperature: 생성될 응답의 다양성(0.0 ~ 1.0) 0.0은 가장 확실한 답변을, 1.0은 가장 다양한 답변을 생성
    stream: 응답을 한 번에 반환할지 여부. False로 설정하면 한 번에 반환
    """
    try:
        response = client.chat.completions.create(
            model = "gpt-4o",
            messages = messages,
            # max_tokens = 150,
            temperature = 0.0,
            stream = False)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [7]:
def extract_raw_text(content):
    """
    JSON 응답에서 raw_text 항목을 추출하는 함수.
    """
    try:
        # JSON 객체에서 raw_text를 추출
        return content.get('raw_text', '')
    except Exception as e:
        print(f"Error extracting raw_text: {e}")
    return ""

def extract_issues(content):
    """
    JSON 응답에서 issue 항목을 추출하는 함수.
    """
    try:
        # JSON 객체에서 issues를 추출
        return content.get('issues', [])
    except Exception as e:
        print(f"Error extracting issues: {e}")
    return []

def extract_sentiments(content):
    """
    JSON 응답에서 각 issue에 대한 sentiment 값을 추출하는 함수.
    """
    try:
        # JSON 객체에서 sentiments를 추출
        return content.get('sentiments', {})
    except Exception as e:
        print(f"Error extracting sentiments: {e}")
    return {}

def extract_sentiment_all(content):
    """
    JSON 응답에서 sentiment_all 값을 추출하는 함수.
    """
    try:
        # JSON 객체에서 sentiment_all 값을 추출
        return content.get('sentiment_all', '')
    except Exception as e:
        print(f"Error extracting sentiment(all): {e}")
    return ""

# 문자열을 딕셔너리로 변환하는 함수
def convert_to_dict(content):
    try:
        # JSON 형식의 문자열을 딕셔너리로 변환
        return json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

In [27]:
def map_issues_to_themes(issues_list, clustering_response):
    themes = []
    for issues in issues_list:
        issue_themes = set()  # 중복을 방지하기 위해 set 사용
        for issue in issues:
            for theme, issue_group in clustering_response.items():
                if issue in issue_group:
                    issue_themes.add(theme)
        themes.append(", ".join(issue_themes))  # 여러 테마가 있을 경우 ","로 결합
    return themes

# 후처리 함수 정의(리스트, 딕셔너리 형태를 문자열로 변환)
def clean_format(value):
    if isinstance(value, list):
        return ', '.join(value)  # 리스트의 경우 ,로 구분하여 문자열로 변환
    if isinstance(value, dict):
        return ', '.join([f"{k}: {v}" for k, v in value.items()])  # 딕셔너리의 경우 key: value 형식으로 변환
    return value  # 이미 문자열인 경우는 그대로 반환


# 파일 읽기

# test 영역

In [9]:
instructions = config['gptapi']['sentiment']['instructions']
few_shot_examples = config['gptapi']['sentiment']['few_shot_examples']
theme_instructions = config['gptapi']['theme']['instructions']

In [10]:
# 처리할 데이터
df = pd.read_csv('data/제공/insight_Paprika Story_20240603.csv')
df.head()

,date,tid,raw_text,theme,issue,mention,sentiment,rating_score,product,data_origin,contact_name
0,2024-05-27 13:05:13,FB-830,잘대응해주시었습니다~^^수고하세요,고객 서비스,NaN,잘대응해주시었습니다수고하세요.,VERY_POSITIVE,5,스탠바이미고,코와샵,ash_****
1,2024-05-27 13:05:01,FB-693,빠른배송 맘에 듭니다. 기사님이 친절하세요.,고객 서비스,친절한 판매자 및 설치 기사,기사님이 친절하세요.,VERY_POSITIVE,5,스탠바이미고,롯데ON,e*l*****
2,2024-05-27 13:04:55,FB-535,제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...,고객 서비스,친절한 판매자 및 설치 기사,친절하시고,VERY_POSITIVE,5,스탠바이미고,옥션,c*o*****
3,2024-05-27 13:04:55,FB-563,빠른 배송과 아울러 문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다! 매우 ...,고객 서비스,빠르고 친절한 응답,문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다,VERY_POSITIVE,5,스탠바이미고,옥션,s*i*****
4,2024-05-27 13:05:01,FB-685,"판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요! 제품도 좋네요 굿굿",고객 서비스,일반적인 서비스 만족도,"판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요!",VERY_POSITIVE,5,스탠바이미고,옥션,d*c*****


## test 1

In [11]:
test = df['raw_text'][:3]
print(test)

0                                   잘대응해주시었습니다~^^수고하세요
1                             빠른배송 맘에 듭니다. 기사님이 친절하세요.
2    제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...
Name: raw_text, dtype: object


In [64]:
text = test[0]
input_raw_text = text
system_message = {"role": "system", "content": instructions}
user_message = {"role": "user", "content": f"{few_shot_examples}\n<raw_text>\n{input_raw_text}\n</raw_text>"}
messages = [system_message, user_message]
content = get_response(messages)
print(content)

{
  "raw_text": "잘대응해주시었습니다~^^수고하세요",
  "issues": [
    "고객 서비스 만족"
  ],
  "sentiments": {
    "고객 서비스 만족": "긍정"
  },
  "sentiment_all": "긍정",
  "theme": [
    "고객 서비스"
  ]
}


In [65]:
content_dict = convert_to_dict(content)
print(content_dict)

{'raw_text': '잘대응해주시었습니다~^^수고하세요', 'issues': ['고객 서비스 만족'], 'sentiments': {'고객 서비스 만족': '긍정'}, 'sentiment_all': '긍정', 'theme': ['고객 서비스']}


In [66]:
raw_text = extract_raw_text(content_dict)
issue = extract_issues(content_dict)
sentiment = extract_sentiments(content_dict)
sentiment_all = extract_sentiment_all(content_dict)
print(raw_text)
print("-------------------")
print(issue)
print("-------------------")
print(sentiment)
print("-------------------")
print(sentiment_all)

잘대응해주시었습니다~^^수고하세요
-------------------
['고객 서비스 만족']
-------------------
{'고객 서비스 만족': '긍정'}
-------------------
긍정


## test 2

In [12]:
test = df['raw_text'][:10]

In [13]:
# 결과를 저장할 리스트 초기화
raw_texts_list = []
issues_list = []
sentiments_list = []
sentiment_all_list = []
themes_list = []

total_iterations = len(test)

with tqdm(total=total_iterations) as pbar:
    for i, text in enumerate(test):

        # (Step 1) 각 raw_text에 대해 issue와 sentiment 추출
        # prompt 생성
        input_raw_text = text
        system_message = {"role": "system", "content": instructions}
        user_message = {"role": "user", "content": f"{few_shot_examples}\n<raw_text>\n{input_raw_text}\n</raw_text>"}
        messages = [system_message, user_message]

        # GPT-4o로부터 응답 받기
        content = get_response(messages)
        if not content:
            pbar.update(1)
            time.sleep(1)
            continue
        
        # 응답 파싱
        content_dict = convert_to_dict(content)
        raw_text = extract_raw_text(content_dict)
        issue = extract_issues(content_dict)
        sentiment = extract_sentiments(content_dict)
        sentiment_all = extract_sentiment_all(content_dict)
        
        # 결과 저장
        raw_texts_list.append(raw_text)
        issues_list.append(issue)
        sentiments_list.append(sentiment)
        sentiment_all_list.append(sentiment_all)
        
        pbar.update(1)
        time.sleep(1)


100%|██████████| 10/10 [00:30<00:00,  3.04s/it]


In [14]:
# 중간 확인
result = pd.DataFrame({
    "raw_text": raw_texts_list,
    "issues": issues_list,
    "sentiments": sentiments_list,
    "sentiment_all": sentiment_all_list,
})
result

,raw_text,issues,sentiments,sentiment_all
0,잘대응해주시었습니다~^^수고하세요,[고객 서비스],{'고객 서비스': '긍정'},긍정
1,빠른배송 맘에 듭니다. 기사님이 친절하세요.,"[빠른 배송, 친절한 기사]","{'빠른 배송': '긍정', '친절한 기사': '긍정'}",긍정
2,제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...,"[좋은 제품, 빠른 배송, 친절한 서비스, 구매확정 늦음]","{'좋은 제품': '긍정', '빠른 배송': '긍정', '친절한 서비스': '긍정'...",긍정
3,빠른 배송과 아울러 문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다! 매우 ...,"[빠른 배송, 친절한 답변, 만족]","{'빠른 배송': '긍정', '친절한 답변': '긍정', '만족': '긍정'}",긍정
4,"판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요! 제품도 좋네요 굿굿","[친절한 판매자, 친절한 기사, 제품 만족]","{'친절한 판매자': '긍정', '친절한 기사': '긍정', '제품 만족': '긍정'}",긍정
5,품질좋고 기사님아주 친절하십니다^^^,"[품질 좋음, 친절한 기사]","{'품질 좋음': '긍정', '친절한 기사': '긍정'}",긍정
6,문의드리면 답변도 빠르시고 물건도하자없이 왔어요 감사합니다!,"[빠른 답변, 하자 없는 제품]","{'빠른 답변': '긍정', '하자 없는 제품': '긍정'}",긍정
7,주문한지 40일만에 물건받았습니다. 중간에 취소하고 다른 곳에서 주문할까 했지만 제...,"[배송 지연, 가격 경쟁력, 판매자 응대, 친절한 설치 기사, 제품 만족]","{'배송 지연': '부정', '가격 경쟁력': '긍정', '판매자 응대': '중립'...",긍정
8,판매자님 상담도 친절하게 해주셨는데 배송도 바로 해주셔서 주문하고 다음날인가? 하루...,"[친절한 상담, 빠른 배송, 높은 활용도, 저렴한 가격]","{'친절한 상담': '긍정', '빠른 배송': '긍정', '높은 활용도': '긍정'...",긍정
9,배송도 빠르고 친절합니다,"[빠른 배송, 친절한 서비스]","{'빠른 배송': '긍정', '친절한 서비스': '긍정'}",긍정


In [21]:
# (Step 2) 모든 issue 데이터를 기반으로 theme 추출 
# 모든 issue를 리스트로 합침
all_issues = [item for sublist in issues_list for item in sublist]
all_issues_str = ', '.join(all_issues)
system_messages_for_clustering = {"role": "system", "content": theme_instructions}
user_messages_for_clustering = {"role": "user", "content": all_issues_str}
messages_for_clustering = [system_messages_for_clustering, user_messages_for_clustering]
themes_response = get_response(messages_for_clustering)
themes_response_dict = convert_to_dict(themes_response)
if themes_response_dict is not None:
    themes = map_issues_to_themes(issues_list, themes_response_dict)
else:
    print("Failed to convert themes response to dictionary.")

In [22]:
print(themes)

['서비스 관련', '서비스 관련, 배송 관련', '제품 품질 관련, 서비스 관련, 구매 절차 관련, 배송 관련', '제품 품질 관련, 서비스 관련, 배송 관련', '제품 품질 관련, 서비스 관련', '제품 품질 관련, 서비스 관련', '제품 품질 관련, 구매 절차 관련', '가격 관련, 제품 품질 관련, 서비스 관련, 배송 관련', '가격 관련, 제품 품질 관련, 서비스 관련, 배송 관련', '서비스 관련, 배송 관련']


In [23]:
result = pd.DataFrame({
    "raw_text": raw_texts_list,
    "issues": issues_list,
    "sentiments": sentiments_list,
    "sentiment_all": sentiment_all_list,
    "themes": themes
})

In [24]:
result

,raw_text,issues,sentiments,sentiment_all,themes
0,잘대응해주시었습니다~^^수고하세요,[고객 서비스],{'고객 서비스': '긍정'},긍정,서비스 관련
1,빠른배송 맘에 듭니다. 기사님이 친절하세요.,"[빠른 배송, 친절한 기사]","{'빠른 배송': '긍정', '친절한 기사': '긍정'}",긍정,"서비스 관련, 배송 관련"
2,제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...,"[좋은 제품, 빠른 배송, 친절한 서비스, 구매확정 늦음]","{'좋은 제품': '긍정', '빠른 배송': '긍정', '친절한 서비스': '긍정'...",긍정,"제품 품질 관련, 서비스 관련, 구매 절차 관련, 배송 관련"
3,빠른 배송과 아울러 문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다! 매우 ...,"[빠른 배송, 친절한 답변, 만족]","{'빠른 배송': '긍정', '친절한 답변': '긍정', '만족': '긍정'}",긍정,"제품 품질 관련, 서비스 관련, 배송 관련"
4,"판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요! 제품도 좋네요 굿굿","[친절한 판매자, 친절한 기사, 제품 만족]","{'친절한 판매자': '긍정', '친절한 기사': '긍정', '제품 만족': '긍정'}",긍정,"제품 품질 관련, 서비스 관련"
5,품질좋고 기사님아주 친절하십니다^^^,"[품질 좋음, 친절한 기사]","{'품질 좋음': '긍정', '친절한 기사': '긍정'}",긍정,"제품 품질 관련, 서비스 관련"
6,문의드리면 답변도 빠르시고 물건도하자없이 왔어요 감사합니다!,"[빠른 답변, 하자 없는 제품]","{'빠른 답변': '긍정', '하자 없는 제품': '긍정'}",긍정,"제품 품질 관련, 구매 절차 관련"
7,주문한지 40일만에 물건받았습니다. 중간에 취소하고 다른 곳에서 주문할까 했지만 제...,"[배송 지연, 가격 경쟁력, 판매자 응대, 친절한 설치 기사, 제품 만족]","{'배송 지연': '부정', '가격 경쟁력': '긍정', '판매자 응대': '중립'...",긍정,"가격 관련, 제품 품질 관련, 서비스 관련, 배송 관련"
8,판매자님 상담도 친절하게 해주셨는데 배송도 바로 해주셔서 주문하고 다음날인가? 하루...,"[친절한 상담, 빠른 배송, 높은 활용도, 저렴한 가격]","{'친절한 상담': '긍정', '빠른 배송': '긍정', '높은 활용도': '긍정'...",긍정,"가격 관련, 제품 품질 관련, 서비스 관련, 배송 관련"
9,배송도 빠르고 친절합니다,"[빠른 배송, 친절한 서비스]","{'빠른 배송': '긍정', '친절한 서비스': '긍정'}",긍정,"서비스 관련, 배송 관련"


In [30]:
result_cleaned = result.apply(lambda col: col.map(clean_format) if col.dtype == 'object' else col)
result_cleaned

,raw_text,issues,sentiments,sentiment_all,themes
0,잘대응해주시었습니다~^^수고하세요,고객 서비스,고객 서비스: 긍정,긍정,서비스 관련
1,빠른배송 맘에 듭니다. 기사님이 친절하세요.,"빠른 배송, 친절한 기사","빠른 배송: 긍정, 친절한 기사: 긍정",긍정,"서비스 관련, 배송 관련"
2,제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...,"좋은 제품, 빠른 배송, 친절한 서비스, 구매확정 늦음","좋은 제품: 긍정, 빠른 배송: 긍정, 친절한 서비스: 긍정, 구매확정 늦음: 중립",긍정,"제품 품질 관련, 서비스 관련, 구매 절차 관련, 배송 관련"
3,빠른 배송과 아울러 문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다! 매우 ...,"빠른 배송, 친절한 답변, 만족","빠른 배송: 긍정, 친절한 답변: 긍정, 만족: 긍정",긍정,"제품 품질 관련, 서비스 관련, 배송 관련"
4,"판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요! 제품도 좋네요 굿굿","친절한 판매자, 친절한 기사, 제품 만족","친절한 판매자: 긍정, 친절한 기사: 긍정, 제품 만족: 긍정",긍정,"제품 품질 관련, 서비스 관련"
5,품질좋고 기사님아주 친절하십니다^^^,"품질 좋음, 친절한 기사","품질 좋음: 긍정, 친절한 기사: 긍정",긍정,"제품 품질 관련, 서비스 관련"
6,문의드리면 답변도 빠르시고 물건도하자없이 왔어요 감사합니다!,"빠른 답변, 하자 없는 제품","빠른 답변: 긍정, 하자 없는 제품: 긍정",긍정,"제품 품질 관련, 구매 절차 관련"
7,주문한지 40일만에 물건받았습니다. 중간에 취소하고 다른 곳에서 주문할까 했지만 제...,"배송 지연, 가격 경쟁력, 판매자 응대, 친절한 설치 기사, 제품 만족","배송 지연: 부정, 가격 경쟁력: 긍정, 판매자 응대: 중립, 친절한 설치 기사: ...",긍정,"가격 관련, 제품 품질 관련, 서비스 관련, 배송 관련"
8,판매자님 상담도 친절하게 해주셨는데 배송도 바로 해주셔서 주문하고 다음날인가? 하루...,"친절한 상담, 빠른 배송, 높은 활용도, 저렴한 가격","친절한 상담: 긍정, 빠른 배송: 긍정, 높은 활용도: 긍정, 저렴한 가격: 긍정",긍정,"가격 관련, 제품 품질 관련, 서비스 관련, 배송 관련"
9,배송도 빠르고 친절합니다,"빠른 배송, 친절한 서비스","빠른 배송: 긍정, 친절한 서비스: 긍정",긍정,"서비스 관련, 배송 관련"


In [30]:
result_cleaned.to_excel('result/result_test_20240920_insight_Paprika Story_20240603.xlsx', index=False)

# 적용 영역

## Step 1) 데이터 선택

In [31]:
text_apply = df['raw_text']

In [32]:
len(text_apply)

1446

In [33]:
text_apply.head()

0                                   잘대응해주시었습니다~^^수고하세요
1                             빠른배송 맘에 듭니다. 기사님이 친절하세요.
2    제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...
3    빠른 배송과 아울러 문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다! 매우 ...
4        판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요! 제품도 좋네요 굿굿
Name: raw_text, dtype: object

## Step 2) issue와 sentiment 추출

In [33]:
# 결과를 저장할 리스트 초기화
raw_texts_list = []
issues_list = []
sentiments_list = []
sentiment_all_list = []

# 총 반복 횟수
total_iterations = len(text_apply)

# progress bar 생성
with tqdm(total=total_iterations) as pbar:
    for i, text in enumerate(text_apply):

        # (Step 1) 각 raw_text에 대해 issue와 sentiment 추출
        # prompt 생성
        input_raw_text = text
        system_message = {"role": "system", "content": instructions}
        user_message = {"role": "user", "content": f"{few_shot_examples}\n<raw_text>\n{input_raw_text}\n</raw_text>"}
        messages = [system_message, user_message]

        # GPT-4o로부터 응답 받기
        content = get_response(messages)
        if not content:
            pbar.update(1)
            time.sleep(1)
            continue
        
        # 응답 파싱
        content_dict = convert_to_dict(content)
        raw_text = extract_raw_text(content_dict)
        issue = extract_issues(content_dict)
        sentiment = extract_sentiments(content_dict)
        sentiment_all = extract_sentiment_all(content_dict)
        
        # 결과 저장
        raw_texts_list.append(raw_text)
        issues_list.append(issue)
        sentiments_list.append(sentiment)
        sentiment_all_list.append(sentiment_all)
        
        pbar.update(1)
        time.sleep(1)


 48%|████▊     | 695/1446 [37:10<48:40,  3.89s/it]  

## Step 3) theme 생성 & mapping

In [34]:
# (Step 2) 모든 issue 데이터를 기반으로 theme 추출 
# 모든 issue를 리스트로 합침
all_issues = [item for sublist in issues_list for item in sublist]
all_issues_str = ', '.join(all_issues)

# GPT API를 통한 theme 추출
system_messages_for_clustering = {"role": "system", "content": theme_instructions}
user_messages_for_clustering = {"role": "user", "content": all_issues_str}
messages_for_clustering = [system_messages_for_clustering, user_messages_for_clustering]

# GPT로부터 응답 받기
themes_response = get_response(messages_for_clustering)
themes_response_dict = convert_to_dict(themes_response)

# 응답 파싱 후 theme 매핑
if themes_response_dict is not None:
    themes = map_issues_to_themes(issues_list, themes_response_dict)
else:
    print("Failed to convert themes response to dictionary.")

In [35]:
# (Step 3) 결과 저장
result = pd.DataFrame({
    "raw_text": raw_texts_list,
    "issues": issues_list,
    "sentiments": sentiments_list,
    "sentiment_all": sentiment_all_list,
    "themes": themes
})
result_cleaned = result.apply(lambda col: col.map(clean_format) if col.dtype == 'object' else col)
result_cleaned

In [36]:
result_cleaned.to_excel('result/result_전체적용_20240922_insight_Paprika Story_20240603.xlsx', index=False)